#### Using WIN M1 Metatrader 5 data

Implementing Saulo's arbitrage algo

The idea is to train a guided system that learns and ajusts itself with its owns errors.  
That's the idea of a kalman's filter.   
The base for the first decision is the Bollinger Bands Algo.  
That's the most powerfull algo, based on the regression toward the mean law!

In [1]:
%matplotlib inline 

In [2]:
pwd

'D:\\Users\\andre.ferreira\\Projects\\stocks\\prototypes'

In [3]:
cd ..

D:\Users\andre.ferreira\Projects\stocks


In [4]:
from matplotlib import pyplot as plt
import os
import sys
import platform
import pandas as pd
import numpy as np
import struct
import datetime
import calendar
import time
import argparse
import seaborn as sns
from algos import meta5Ibov
from algos.util import progressbar
from algos.meanreversion import meanrev, view

import scipy
import talib as ta
from numba import jit
import importlib

In [5]:
importlib.reload(meanrev)

<module 'algos.meanreversion.meanrev' from 'D:\\Users\\andre.ferreira\\Projects\\stocks\\algos\\meanreversion\\meanrev.py'>

In [ ]:
cd data

In [ ]:
if platform.uname()[1] == 'andre-predator': 
    meta5Ibov.setDataPath('C:\\Users\\andre\\Projects\\stocks\\data', 
                        'C:\\Users\\andre\\Projects\\stocks\\data')
else:
    meta5Ibov.setDataPath('D:\\Users\\andre.ferreira\\Projects\\stocks\\data', 
                        'D:\\Users\\andre.ferreira\\Projects\\stocks\\data')


#### Using M1 Data but could be M15 data etc.

Saulo used 10 seconds interval

In [ ]:
#meta5Ibov.loadMeta5Data(preload=False);
meta5Ibov.loadMeta5Data();
bars = meta5Ibov.getSymbol('WIN@')

Not enough memory on local machine

In [ ]:
bars = bars[-9000:]

In [ ]:
window=21
nbands=3
amount=1.
targetprofit=15.
stoploss=100.
batchn=60 # number of previous samples or indicators to help find a pattern

In [ ]:
tbars = meanrev.rawSignals(bars, window, nbands)

In [ ]:
tbars.columns

In [ ]:
signal = meanrev.lastSignal(tbars, nbands)

In [ ]:
#if np.all(signal == 0): # no signal in any band no training
#  return signal, None, None, None

In [ ]:
tbars = meanrev.barsFeatured(tbars, window, nbands)

In [ ]:
tbars.columns

In [ ]:
tbars = meanrev.targetFromSignals(tbars, nbands,
        amount, targetprofit, stoploss) # needs day identifier integer

In [ ]:
tbars.columns

In [ ]:
isgfeatures = meanrev.getIndexFeatures(tbars, nbands); # signal features standardized

In [ ]:
isgfeatures[-1], tbars.columns[isgfeatures[-1]]

In [ ]:
nbands, batchn

In [ ]:
X, y, time = meanrev.getTrainingVectors(tbars, isgfeatures, nbands, batchn)

In [ ]:
Xforecast = meanrev.getForecastVector(tbars, isgfeatures, nbands, batchn)

In [ ]:
X.shape, y.shape

In [ ]:
Xforecast.shape

In [ ]:
plt.hist(y);

In [ ]:
view.viewBands(tbars, nbands=nbands, lastn=-5000, str_bandsg='y')

In [ ]:
tbars.columns

In [ ]:
def getTrainingForecastVectors(obars, window=21, nbands=3,
        amount=1, targetprofit=15., stoploss=45., batchn=180):
    """
    return Xpredict, Xtrain, ytrain

    if Xpredict has signal all zero return None
    """
    bars = meanrev.rawSignals(obars, window, nbands)
    signal = meanrev.lastSignal(bars, nbands)
    if np.all(signal == 0): # no signal in any band no training
        return signal, None, None, None
    bars = meanrev.barsFeatured(bars, window, nbands)
    bars = meanrev.targetFromSignals(bars, nbands,
            amount, targetprofit, stoploss) # needs day identifier integer
    isgfeatures = meanrev.getIndexFeatures(bars, nbands); # signal features standardized
    X, y, time = meanrev.getTrainingVectors(bars, isgfeatures, nbands, batchn)
    Xforecast = meanrev.getForecastVector(bars, isgfeatures, nbands, batchn)

    return signal, Xforecast, X, y

In [ ]:
%%time
signal, Xforecast, X, y = getTrainingForecastVectors(bars, window, nbands, amount, targetprofit, stoploss, batchn)
#%%timeit 2.79 s ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
Xforecast.shape

In [ ]:
X.shape, y.shape

### Debugging Meta5

In [6]:
cd "D:\\MetaTrader 5"

D:\MetaTrader 5


In [ ]:
X = np.fromfile('X.npy', dtype=np.float64)
y = np.fromfile('y.npy', dtype=np.int)

In [ ]:
X[ X > 10**300]

In [ ]:
np.argwhere(X > 10**300)[-10:]

In [7]:
#EMPTY_VALUE FROM MT5
np.finfo('d').max 

1.7976931348623157e+308

In [ ]:
X[74969]

In [ ]:
y

In [ ]:
# b'' empty bytes string
X = X.reshape(-1, 1260)